In [1]:
from itertools import product
from tqdm import tqdm
import csv
from data_generation.word import label
import opencc
import os


2022-12-29 19:59:45.081831: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/home/yuan/anaconda3/envs/wtcpt/lib/python3.8/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:903: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/home/yuan/anaconda3/envs/wtcpt/lib/python3.8/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '
2022-12-29 19:59:47.711760: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-12-29 19:59:4

In [2]:
sents_file = 'sents.csv'
sents_ = []
with open(sents_file,'r') as f:
    sents_ = [i for i in csv.reader(f)][1:]

sents = sents_[:200000]

In [3]:
class label_token():
    def __init__(self, sents, max, p, pos_tag):
        
        self.max_length = max
        self.p = p
        self.pos_tag = pos_tag

        self.result = []
        self.generate()

    def label_padding(self, label):
        while len(label) < self.max_length:
            label+=('0')
        return label

    def generate(self):
        
        if not os.path.isdir('tokenLabel'):
            os.mkdir('tokenLabel')

        for sent in tqdm(sents):
            
            try:
                if len(sent) > self.max_length:
                    sent = sent[0:self.max_length-1]

                s_text, s_label = label(sent,self.pos_tag,self.p)
                
                s_label = self.label_padding(s_label)

                if '1' in s_label:
                    self.result.append([s_text,s_label])
                    
            except:
                continue
        
        with open('tokenLabel/tokenLabel_'+str(len(self.result))+'_'+self.pos_tag+'_'+str(self.max_length)+'_'+str(self.p)+'_'+'.csv', 'w', newline='') as f:

            fieldnames = ['text']
            for i in range(self.max_length):
                fieldnames.append(str(i))

            writer = csv.writer(f)
            writer.writerow(fieldnames)

            for i in self.result:
                writer.writerow([i[0]]+list(i[1]))
            

In [ ]:
lengths = [96]
ps = [0.8]
tags = ['A','D','P','Nf','Cbb']

for length,p,tag in product(lengths,ps,tags):
    print(length, p, tag)
    label_token(sents, length, p, tag)

In [ ]:
# lengths = [96]
# ps = [0.5,0.2,0.8]
# tags = ['A','D','P','Nf','Cbb']

# for length,p,tag in product(lengths,ps,tags):
#     print(length, p, tag)
#     label_token(sents, length, p, tag)